In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

data_dir = 'data'

with open(os.path.join(data_dir, 'divorce.csv')) as clean_data_file:
    data = pd.read_csv(clean_data_file, delimiter=';')

In [5]:
print(data.shape)

(170, 55)


In [6]:
print(data[:10])

   Atr1  Atr2  Atr3  Atr4  Atr5  Atr6  Atr7  Atr8  Atr9  Atr10  ...  Atr46  \
0     2     2     4     1     0     0     0     0     0      0  ...      2   
1     4     4     4     4     4     0     0     4     4      4  ...      2   
2     2     2     2     2     1     3     2     1     1      2  ...      3   
3     3     2     3     2     3     3     3     3     3      3  ...      2   
4     2     2     1     1     1     1     0     0     0      0  ...      2   
5     0     0     1     0     0     2     0     0     0      1  ...      2   
6     3     3     3     2     1     3     4     3     2      2  ...      3   
7     2     1     2     2     2     1     0     3     3      2  ...      0   
8     2     2     1     0     0     4     1     3     3      3  ...      1   
9     1     1     1     1     1     2     0     2     2      2  ...      2   

   Atr47  Atr48  Atr49  Atr50  Atr51  Atr52  Atr53  Atr54  Class  
0      1      3      3      3      2      3      2      1      1  
1      

In [12]:
X = data.iloc[:, :-1].to_numpy()
y = data.iloc[:, -1].to_numpy()
print(X[:10])

[[2 2 4 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 2 1 2 0 1
  2 1 3 3 2 1 1 2 3 2 1 3 3 3 2 3 2 1]
 [4 4 4 4 4 0 0 4 4 4 4 3 4 0 4 4 4 4 3 2 1 1 0 2 2 1 2 0 1 1 0 4 2 3 0 2
  3 4 2 4 2 2 3 4 2 2 2 3 4 4 4 4 2 2]
 [2 2 2 2 1 3 2 1 1 2 3 4 2 3 3 3 3 3 3 2 1 0 1 2 2 2 2 2 3 2 3 3 1 1 1 1
  2 1 3 3 3 3 2 3 2 3 2 3 1 1 1 2 2 2]
 [3 2 3 2 3 3 3 3 3 3 4 3 3 4 3 3 3 3 3 4 1 1 1 1 2 1 1 1 1 3 2 3 2 2 1 1
  3 3 4 4 2 2 3 2 3 2 2 3 3 3 3 2 2 2]
 [2 2 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 2 1 1 0 0 0 0 2 1 2 1 1 1 1 1 1 0 0
  0 0 2 1 0 2 3 0 2 2 1 2 3 2 2 2 1 0]
 [0 0 1 0 0 2 0 0 0 1 0 2 1 0 2 0 2 1 0 1 0 0 0 0 2 2 0 0 0 0 4 1 1 1 1 1
  1 2 0 2 2 1 2 3 0 2 2 1 2 1 1 1 2 0]
 [3 3 3 2 1 3 4 3 2 2 2 2 2 3 2 3 3 3 3 2 3 3 3 3 2 3 3 2 2 2 1 2 2 1 1 2
  3 2 2 3 3 3 3 4 3 3 2 3 2 3 3 2 2 2]
 [2 1 2 2 2 1 0 3 3 2 4 3 2 3 4 3 2 3 2 1 2 1 1 2 3 3 2 2 2 3 1 1 0 2 2 1
  4 4 4 4 4 4 3 2 0 0 1 2 2 2 1 1 1 0]
 [2 2 1 0 0 4 1 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 3 2 3 2 3 2 3 1 1 1 1 1 1
  1 2 2 2 2 2 2 2 2 1 

In [14]:
statements = {'1': "If one of us apologizes when our discussion deteriorates, the discussion ends.",
'2': "I know we can ignore our differences, even if things get hard sometimes.",
'3': "When we need it, we can take our discussions with my spouse from the beginning and correct it.",
'4': "When I discuss with my spouse, to contact him will eventually work.",
'5': "The time I spent with my wife is special for us.",
'6': "We don't have time at home as partners.",
'7': "We are like two strangers who share the same environment at home rather than family.",
'8': "I enjoy our holidays with my wife.",
'9': "I enjoy traveling with my wife.",
'10': "Most of our goals are common to my spouse.",
'11': "I think that one day in the future, when I look back, I see that my spouse and I have been in harmony with each other.",
'12': "My spouse and I have similar values in terms of personal freedom.",
'13': "My spouse and I have similar sense of entertainment.",
'14': "Most of our goals for people (children, friends, etc.) are the same.",
'15': "Our dreams with my spouse are similar and harmonious.",
'16': "We're compatible with my spouse about what love should be.",
'17': "We share the same views about being happy in our life with my spouse",
'18': "My spouse and I have similar ideas about how marriage should be",
'19': "My spouse and I have similar ideas about how roles should be in marriage",
'20: "My spouse and I have similar values in trust.",
'21': "I know exactly what my wife likes.",
'22': "I know how my spouse wants to be taken care of when she/he sick.",
'23': "I know my spouse's favorite food.",
'24': "I can tell you what kind of stress my spouse is facing in her/his life.",
'25': "I have knowledge of my spouse's inner world.",
'26': "I know my spouse's basic anxieties.",
'27': "I know what my spouse's current sources of stress are.",
'28': "I know my spouse's hopes and wishes.",
'29': "I know my spouse very well.",
'30': "I know my spouse's friends and their social relationships.",
'31': "I feel aggressive when I argue with my spouse.",
'32': "When discussing with my spouse, I usually use expressions such as ‘you always’ or ‘you never’ .",
'33': "I can use negative statements about my spouse's personality during our discussions.",
'34': "I can use offensive expressions during our discussions.",
'35': "I can insult my spouse during our discussions.",
'36': "I can be humiliating when we discussions.",
'37': "My discussion with my spouse is not calm.",
'38': "I hate my spouse's way of open a subject.",
'39': "Our discussions often occur suddenly.",
'40': "We're just starting a discussion before I know what's going on.",
'41': "When I talk to my spouse about something, my calm suddenly breaks.",
'42': "When I argue with my spouse, ı only go out and I don't say a word.",
'43': "I mostly stay silent to calm the environment a little bit.",
'44': "Sometimes I think it's good for me to leave home for a while.",
'45': "I'd rather stay silent than discuss with my spouse.",
'46': "Even if I'm right in the discussion, I stay silent to hurt my spouse.",
'47': "When I discuss with my spouse, I stay silent because I am afraid of not being able to control my anger.",
'48': "I feel right in our discussions.",
'49': "I have nothing to do with what I've been accused of.",
'50': "I'm not actually the one who's guilty about what I'm accused of.",
'51': "I'm not the one who's wrong about problems at home.",
'52': "I wouldn't hesitate to tell my spouse about her/his inadequacy.",
'53': "When I discuss, I remind my spouse of her/his inadequacy.",
'54': "I'm not afraid to tell my spouse about her/his incompetence."} 
print(statements['1'])

SyntaxError: EOL while scanning string literal (<ipython-input-14-27730fe3298e>, line 6)